In [1]:
from agent_configs import RainbowConfig
import gymnasium as gym
import torch
import random
import numpy as np
import torch
from utils import CategoricalCrossentropyLoss, KLDivergenceLoss
from utils.utils import HuberLoss
from cfr_utils import EvalWrapper, evaluatebots, WrapperEnv, load_agents, EmptyConf, NFSPWrapper, NFSPEvalWrapper, LoadNFSPAgent
import pyspiel
import copy
from agent_configs.cfr_config import CFRConfig
from active_player import ActivePlayer
from cfr_agent import CFRAgent
from cfr_network import CFRNetwork
import sys
sys.path.append("..")
from dqn.rainbow.rainbow_agent import RainbowAgent
import tensorflow as tf
import os
import open_spiel.python.algorithms.nfsp

tf.config.set_visible_devices([], 'GPU')
num_players = 2
max_nodes = 10000000

fhp = pyspiel.load_game("universal_poker", {"numPlayers":2, "numSuits": 4, "numRanks":13, "numHoleCards": 2, "numBoardCards": "0 3", "bettingAbstraction": "fcpa", "numRounds":2, "blind": "50 100"})
leduc = pyspiel.load_game("universal_poker", {"numPlayers":2, "numSuits": 2, "numRanks":3, "numHoleCards": 1, "numBoardCards": "0 1", "bettingAbstraction": "fcpa", "numRounds":2, "blind": "50 100"})
leducconfig = {"state_representation_size": 16}
fhpconfig = {"state_representation_size": 108}
leducgame = NFSPWrapper(leduc)
fhpgame = NFSPWrapper(fhp)
chosen_game = 'leduc'

active_player_obj = ActivePlayer(2)

config_dict = {
    "dense_layer_widths": [128, 128],
    "value_hidden_layer_widths": [128],
    "advatage_hidden_layer_widths": [128],
    "adam_epsilon": 1e-8,
    "learning_rate": 0.002,
    "training_steps": 1000000,
    "per_epsilon": 0.0001,
    "per_alpha": 0,
    "per_beta": 0,
    "minibatch_size": 256,
    "replay_buffer_size": 100000,
    "min_replay_buffer_size": 256,
    "transfer_interval": 1280,
    "n_step": 1,
    "loss_function": HuberLoss(),  # could do categorical cross entropy
    "clipnorm": 0.0,
    "discount_factor": 0.99,
    "atom_size": 1,
    "replay_interval": 64,
    "dueling": True,
    "noisy_sigma": 0.0,
    "eg_epsilon": 1,
    "eg_epsilon_final": 0.0,
    "eg_epsilon_final_step": 2000,
    "eg_epsilon_decay_type": "linear",
    "num_minibatches": 4,
}
gameconfig = EmptyConf()
config = RainbowConfig(config_dict, gameconfig)
device = "cuda:0" if torch.cuda.is_available() else "cpu"
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
tf.compat.v1.disable_v2_behavior()


mainpath = 'nfspfilecheckpoints/leduc/nfsp/0/10000000/'
nodes = 0
games = [leducgame]
for i in games:
    i.reset()
    if i == leducgame:
        game_string = "leduc"
    else:
        game_string = "fhp"
    with tf.compat.v1.Session() as sess:
            agent = open_spiel.python.algorithms.nfsp.NFSP(
            session=sess,
            player_id=0,
            state_representation_size=leducconfig["state_representation_size"] if i == leducgame else fhpconfig["state_representation_size"],
            num_actions=4,
            hidden_layers_sizes=[1024,512,1024,512],
            reservoir_buffer_capacity=30000000,
            anticipatory_param=0,
            batch_size=256, rl_learning_rate=0.1, sl_learning_rate=0.01, min_buffer_size_to_learn=1000, learn_every=256, optimizer_str="sgd", replay_buffer_capacity= 600000, epsilon_start=0.08, epsilon_end=0,
            )
            LoadNFSPAgent(mainpath, agent, 0)
            agent.restore(mainpath) # IF YOU HAVE A NFSP AGENT PATH
            # agent.restore(path1) # IF YOU HAVE A NFSP AGENT PATH
            sess.run(tf.compat.v1.global_variables_initializer())

            print(agent)
            wrapped = NFSPEvalWrapper(leducgame, agent, 16, 4)

            evaluator = RainbowAgent(wrapped, config, name="Rainbow-Leduc-Test-NFSP", device=device)
            evaluator.checkpoint_interval = 100

            for param in evaluator.model.parameters():
                print(param)
            evaluator.train()





/Users/Ile-Maurice/Library/Python/3.10/lib/python/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


Instructions for updating:
non-resource variables are not supported in the long term
Using default save_intermediate_weights     : False
Using         training_steps                : 1000000
Using         adam_epsilon                  : 1e-08
Using default momentum                      : 0.9
Using         learning_rate                 : 0.002
Using         clipnorm                      : 0.0
Using default optimizer                     : <class 'torch.optim.adam.Adam'>
Using default weight_decay                  : 0.0
Using         loss_function                 : <utils.utils.HuberLoss object at 0x107772ef0>
Using default activation                    : relu
Using         kernel_initializer            : None
Using         minibatch_size                : 256
Using         replay_buffer_size            : 100000
Using         min_replay_buffer_size        : 256
Using         num_minibatches               : 4
Using default training_iterations           : 1
Using default print_interval      

2025-04-24 12:15:38.639602: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled


nfspfilecheckpoints/leduc/nfsp/0/10000000/q_network_pid0
save_4/restore_all
INFO:tensorflow:Restoring parameters from nfspfilecheckpoints/leduc/nfsp/0/10000000/avg_network_pid0
nfspfilecheckpoints/leduc/nfsp/0/10000000/avg_network_pid0
save_5/restore_all
num_actions:  4


2025-04-24 12:15:38.896683: W tensorflow/c/c_api.cc:305] Operation '{name:'mlp_8/weights_4/Assign' id:918 op device:{requested: '', assigned: ''} def:{{{node mlp_8/weights_4/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](mlp_8/weights_4, zeros_19)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


float32
Parameter containing:
tensor([[-0.0044, -0.1243,  0.1913,  ...,  0.0255, -0.1001,  0.2298],
        [ 0.1996, -0.0481, -0.0778,  ...,  0.0945, -0.2263, -0.2027],
        [-0.0340, -0.1023,  0.1091,  ...,  0.1997,  0.0090, -0.0223],
        ...,
        [-0.1736,  0.2160, -0.0177,  ...,  0.0378,  0.1750,  0.1864],
        [-0.1859,  0.0659, -0.1290,  ...,  0.0348, -0.1870, -0.0815],
        [ 0.0036, -0.0012, -0.1210,  ...,  0.1428, -0.0432, -0.1808]],
       requires_grad=True)
Parameter containing:
tensor([ 0.1618,  0.0498,  0.0314,  0.2443,  0.0833, -0.1522, -0.0083, -0.0903,
         0.0653,  0.0690, -0.1450,  0.0424, -0.1346, -0.1656, -0.1687,  0.1819,
        -0.1800, -0.0604,  0.0692, -0.1614,  0.0260, -0.2305, -0.2426, -0.2352,
         0.0985,  0.2212, -0.0407,  0.1353,  0.0111, -0.1273,  0.0705,  0.2068,
         0.0297, -0.2158,  0.0600,  0.0711, -0.1060,  0.1775, -0.0322,  0.0232,
        -0.0598,  0.1600,  0.1821,  0.1500,  0.1876, -0.0296,  0.0355, -0.2391,
       

KeyboardInterrupt: 